# MLOps Zoomcamp: Week 1 Homework

The assign for [week 1](https://github.com/DataTalksClub/mlops-zoomcamp/tree/main/01-intro) is to explore the [New York City taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Taxi drivers in New York need to have a permit to conduct their business. Yellow taxis carry a taxi medallion, which acts as their permit.  They are free to conduct their business throughout New York City.  Green taxis have a separate permit, but they are restricted to operate in certain areas.

This assignment uses the Yellow Taxi Trip records from January and February 2023.  

Sources:
- [CityLimits](https://citylimits.org/2019/11/30/need-to-know-taxi-medallions-in-new-york/)
- [Wikipedia](https://en.wikipedia.org/wiki/Taxis_of_New_York_City)

In [1]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, root_mean_squared_error

In [2]:
df_jan = pd.read_parquet('yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('yellow_tripdata_2023-02.parquet')

In [3]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
df_feb.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [5]:
df_jan.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [6]:
df_feb.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')

In [7]:
# Question 1: How many columns are there in the dataset?
f"{len(df_jan.columns)} columns"

'19 columns.'

In [8]:
df_jan['trip_duration'] = df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']
df_feb['trip_duration'] = df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']

In [9]:
df_jan['trip_duration']

0         0 days 00:08:26
1         0 days 00:06:19
2         0 days 00:12:45
3         0 days 00:09:37
4         0 days 00:10:50
                ...      
3066761   0 days 00:13:59
3066762   0 days 00:19:27
3066763   0 days 00:24:31
3066764   0 days 00:13:00
3066765   0 days 00:14:24
Name: trip_duration, Length: 3066766, dtype: timedelta64[us]

In [10]:
def get_column_stats(col):
    col_std = col.std()
    col_mean = col.mean()
    col_median = col.median()
    print(f"Standard Deviation: {col_std}")
    print(f"Mean: {col_mean}")
    print(f"Media: {col_median}")
    return col_std, col_mean, col_median

In [11]:
# Question 2: What's the standard deviation of the trips duration in January?
get_column_stats(df_jan['trip_duration'])

Standard Deviation: 0 days 00:42:35.661074
Mean: 0 days 00:15:40.139710
Media: 0 days 00:11:31


(Timedelta('0 days 00:42:35.661074'),
 Timedelta('0 days 00:15:40.139710'),
 Timedelta('0 days 00:11:31'))

In [12]:
get_column_stats(df_feb['trip_duration'])

Standard Deviation: 0 days 00:42:50.526105
Mean: 0 days 00:16:00.954577
Media: 0 days 00:11:48


(Timedelta('0 days 00:42:50.526105'),
 Timedelta('0 days 00:16:00.954577'),
 Timedelta('0 days 00:11:48'))

In [13]:
# Sorted timestamps
df_jan['trip_duration'].sort_values(ascending=True)

1839180   -1 days +23:30:48
2218504   -1 days +23:40:48
2124414   -1 days +23:45:28
1188757     0 days 00:00:00
2575201     0 days 00:00:00
                 ...       
894362      2 days 01:07:47
553536      2 days 02:40:31
2543477     2 days 22:20:19
1424460     4 days 06:59:24
2127658     6 days 23:09:11
Name: trip_duration, Length: 3066766, dtype: timedelta64[us]

In [14]:
# Sorted minutes
df_jan['trip_duration'].sort_values(ascending=True).dt.total_seconds()/60

1839180      -29.200000
2218504      -19.200000
2124414      -14.533333
1188757        0.000000
2575201        0.000000
               ...     
894362      2947.783333
553536      3040.516667
2543477     4220.316667
1424460     6179.400000
2127658    10029.183333
Name: trip_duration, Length: 3066766, dtype: float64

In [15]:
def filter_df_by_trip_duration(df):
    df['trip_duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['trip_duration_minutes'] = df['trip_duration'].apply(lambda td: td.total_seconds() / 60)
    filtered_df = df[(df['trip_duration_minutes'] >= 1) & (df['trip_duration_minutes'] <= 60)]
    return filtered_df

In [16]:
filtered_df_jan = filter_df_by_trip_duration(df_jan)
filtered_df_feb = filter_df_by_trip_duration(df_feb)

In [17]:
filtered_df_jan['trip_duration_minutes'].sort_values(ascending=True)

2995384     1.0
3045964     1.0
2283193     1.0
3063488     1.0
2808159     1.0
           ... 
3020494    60.0
3056316    60.0
3065986    60.0
2949108    60.0
718916     60.0
Name: trip_duration_minutes, Length: 3009173, dtype: float64

In [18]:
# Question 3: What fraction of the records were left after you dropped the outliers?
len(filtered_df_jan) / len(df_jan) * 100

98.1220282212598

In [19]:
# Convert location ids to strings so they won't be used as numbers
# One-hot encoding needs categorical values
filtered_df_jan.loc[:, 'PULocationID'] = filtered_df_jan['PULocationID'].astype(str)
filtered_df_jan.loc[:, 'DOLocationID'] = filtered_df_jan['DOLocationID'].astype(str)
filtered_df_feb.loc[:, 'PULocationID'] = filtered_df_feb['PULocationID'].astype(str)
filtered_df_feb.loc[:, 'DOLocationID'] = filtered_df_feb['DOLocationID'].astype(str)

/tmp/ipykernel_98107/11292547.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['161' '43' '48' ... '114' '230' '262']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  filtered_df_jan.loc[:, 'PULocationID'] = filtered_df_jan['PULocationID'].astype(str)
/tmp/ipykernel_98107/11292547.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['141' '237' '238' ... '239' '79' '143']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  filtered_df_jan.loc[:, 'DOLocationID'] = filtered_df_jan['DOLocationID'].astype(str)
/tmp/ipykernel_98107/11292547.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['142' '132' '161' ... '158' '79' '161']' has dtype incompatible with int32, please explicitly cast to a co

In [20]:
categorical = ['PULocationID', 'DOLocationID']
train_dicts = filtered_df_jan[categorical].to_dict(orient='records')
print(train_dicts[:10])

[{'PULocationID': '161', 'DOLocationID': '141'}, {'PULocationID': '43', 'DOLocationID': '237'}, {'PULocationID': '48', 'DOLocationID': '238'}, {'PULocationID': '138', 'DOLocationID': '7'}, {'PULocationID': '107', 'DOLocationID': '79'}, {'PULocationID': '161', 'DOLocationID': '137'}, {'PULocationID': '239', 'DOLocationID': '143'}, {'PULocationID': '142', 'DOLocationID': '200'}, {'PULocationID': '164', 'DOLocationID': '236'}, {'PULocationID': '141', 'DOLocationID': '107'}]


In [45]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [46]:
target = 'trip_duration_minutes'
y_train = filtered_df_jan[target].values

In [47]:
# Question 4: What is the dimensionality of the matrix?
X_train.shape

(3009173, 515)

In [48]:
X_train[:10]

<10x515 sparse matrix of type '<class 'numpy.float64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [49]:
feature_names = dv.get_feature_names_out()
feature_names[0:10]

array(['DOLocationID=1', 'DOLocationID=10', 'DOLocationID=100',
       'DOLocationID=101', 'DOLocationID=102', 'DOLocationID=106',
       'DOLocationID=107', 'DOLocationID=108', 'DOLocationID=109',
       'DOLocationID=11'], dtype=object)

In [50]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [51]:
# Question 4: What is the RMSE on the training data?
y_pred = lr.predict(X_train)

In [52]:
mean_squared_error(y_train, y_pred, squared=False)

/home/greg/code/study/mlops-course/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.649261937621321

In [53]:
root_mean_squared_error(y_train, y_pred)

7.649261937621321

In [54]:
val_dicts = filtered_df_feb[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = filtered_df_feb[target].values

In [55]:
y_val

array([ 1.68333333, 32.08333333, 13.3       , ..., 14.        ,
        7.        ,  9.8       ])

In [56]:
y_pred = lr.predict(X_val)

In [57]:
y_pred

array([11.36779004, 49.69482891, 15.1879846 , ..., 13.53318393,
       11.09561621, 13.15701147])

In [58]:
# Question 5: What is the RMSE on the test data?
root_mean_squared_error(y_val, y_pred)

7.811817646307258